In [ ]:
#
# Summary:
#

#
# This Python script processes audio files stored in Microsoft Azure cloud storage by removing background music, leaving only the vocal parts.
# Here's how it works:
#
# 1. Downloads audio files from a cloud storage container.
# 2. Cleans each file using an AI tool called Demucs to isolate vocals and discard instrumental parts.
# 3. Uploads the cleaned vocal-only audio files back to another cloud storage container.
# 4. Cleans up any temporary files created during the process.
#
# It also includes retry logic to handle occasional failures during file download or upload.
#
# 
# This Notebook requires the following environment variables (.env file):
# 

# CLEAN_AUDIO_LOCAL_WORKING_DIR=TODO
# CLEAN_AUDIO_INPUT_CONTAINER_SAS_URI=TODO
# CLEAN_AUDIO_OUTPUT_CONTAINER_SAS_URI=TODO

In [ ]:
import os
import shutil
import subprocess
import time
import uuid
from azure.storage.blob import BlobBlock, BlobClient, ContainerClient
from dotenv import load_dotenv

MAX_RETRIES = 3
BASE_DELAY = 1

def clean_audio(source: str, dest: str):
    for file_name in os.listdir(source):
        source_path = os.path.join(source, file_name)

        subprocess.run(["demucs","--two-stems", "vocals", "--out", dest, source_path], check=True)

        base_name = os.path.splitext(file_name)[0]
        demucs_folder = os.path.join(dest, "htdemucs", base_name)
        vocals_path = os.path.join(demucs_folder, "vocals.wav")

        if os.path.exists(vocals_path):
            flat_vocals_path = os.path.join(dest, f"{base_name}_clean.wav")
            shutil.move(vocals_path, flat_vocals_path)
            shutil.rmtree(os.path.join(dest, "htdemucs"))        

def download_blobs(client: ContainerClient, dest: str):
    blobs = list(client.list_blobs())
    for blob in blobs:
        local_blob_path = os.path.join(dest, blob.name)
        for attempt in range(1, MAX_RETRIES + 1):
            try: 
                with open(local_blob_path, "wb") as file:
                    stream = client.download_blob(blob.name)
                    for chunk in stream.chunks():
                        file.write(chunk)
                        
                break
            except Exception as e:
                if attempt == MAX_RETRIES:
                    raise
                else:
                    print(f"download_blobs() error: {str(e)}")
                    delay = BASE_DELAY * (2 ** (attempt -1))
                    time.sleep(delay)                    

def get_configuration() -> dict[str, str]:
    return {
        "local_working_dir": os.path.normpath(os.getenv('CLEAN_AUDIO_LOCAL_WORKING_DIR')),
        "input_container_sas": os.getenv('CLEAN_AUDIO_INPUT_CONTAINER_SAS_URI'),
        "output_container_sas": os.getenv('CLEAN_AUDIO_OUTPUT_CONTAINER_SAS_URI')
    }

def get_container_client_from_sas(sas_uri: str) -> ContainerClient:
    return ContainerClient.from_container_url(sas_uri)

def get_temp_dir(path: str) -> str:
    dir = os.path.join(path, str(uuid.uuid4()))
    os.makedirs(dir, exist_ok=True)
    return dir

def remove_temp_dir(path: str):
    shutil.rmtree(path)

def upload_blobs(client: ContainerClient, source: str):
    for file_name in os.listdir(source):
        local_file_path = os.path.join(source, file_name)
        if not os.path.isfile(local_file_path):
            continue
        
        blob_client = client.get_blob_client(file_name)

        for attempt in range(1, MAX_RETRIES + 1):
            try:
                block_list = []
                chunk_size = 4 * 1024 * 1024

                with open(local_file_path, "rb") as file:
                    while True:
                        chunk = file.read(chunk_size)
                        if not chunk:
                            break
                        block_id = str(len(block_list)).zfill(6)
                        blob_client.stage_block(block_id=block_id, data=chunk)
                        block_list.append(BlobBlock(block_id=block_id))

                blob_client.commit_block_list(block_list)                
                break
            except Exception as e:
                if attempt == MAX_RETRIES:
                    raise
                else:
                    print(f"upload_blobs() error: {str(e)}")
                    delay = BASE_DELAY * (2 ** (attempt - 1))
                    time.sleep(delay)

def main():
    print("Notebook Cell Running...")

    print("Loading configuration...")
    load_dotenv()
    configuration = get_configuration()

    print("Creating local input and output directories...")
    local_input_dir = get_temp_dir(configuration["local_working_dir"])
    local_output_dir = get_temp_dir(configuration["local_working_dir"])

    print("Obtaining input and output container clients...")
    input_client = get_container_client_from_sas(configuration['input_container_sas'])
    output_client = get_container_client_from_sas(configuration['output_container_sas'])

    print("Downloading input files locally...")
    download_blobs(input_client, local_input_dir)

    print("Cleaning audio...")
    clean_audio(local_input_dir, local_output_dir)

    print("Uploading output files to cloud...")
    upload_blobs(output_client, local_output_dir)

    print("Cleaning local input and output directories...")
    remove_temp_dir(local_input_dir)
    remove_temp_dir(local_output_dir)

    print("Done!")

main()